![sslogo](https://github.com/stratascratch/stratascratch.github.io/raw/master/assets/sslogo.jpg)

# Combining Data for Analysis (Joining/Merging Dataframes)

- https://pandas.pydata.org/pandas-docs/stable/merging.html

In this lesson we will use 3 datasets provided by Yelp.
* yelp_business
* yelp_business_hours
* yelp_reviews

In [1]:
!pip install psycopg2-binary
import pandas as pd
import numpy  as np
import psycopg2 as ps

     |████████████████████████████████| 2.9MB 2.9MB/s 


In [42]:
# Connect to database
host_name = 'db-strata.stratascratch.com'
dbname = 'db_strata'
port = '5432'
# TODO Write your username and database password in the following two variables
user_name = 'ankit082006' #enter username
pwd = 'ofkMQPktC' #enter your database password found in the profile tab in Strata Scratch

try:
    conn = ps.connect(host=host_name,database=dbname,user=user_name,password=pwd,port=port)
except ps.OperationalError as e:
    raise e
else:
    print('Connected!')

Connected!


In [0]:
# Get the data we will use for this exercise

def get_dataset(dataset_name):
    #Write SQL below to pull datasets 
    cur = conn.cursor()
    cur.execute(""" 
                SELECT *  FROM {0}; 
                """.format(dataset_name))
    data = cur.fetchall()
    colnames = [desc[0] for desc in cur.description] 
    conn.commit()

    #create the pandas dataframe
    dataframe = pd.DataFrame(data, columns=colnames)

    #close the connection
    cur.close()
    
    return dataframe

yelp_business = get_dataset("yelp_business")
yelp_hours    = get_dataset("yelp_business_hours")
yelp_reviews  = get_dataset("yelp_reviews")

# a little bit preprocessing
yelp_reviews['business_name'] = yelp_reviews['business_name'].apply(lambda x: '"{0}"'.format(x))

In [4]:
yelp_business.head()

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
0,FYWN1wneV18bWNgQjJ2GNg,Dental by Design,None,"4855 E Warner Rd, Ste B9",Ahwatukee,AZ,85044,33.330690,-111.978599,4.0,22,1,Dentists;General Dentistry;Health & Medical;Or...
1,He-G7vWjzVUysIKrfNbPUQ,Stephen Szabo Salon,None,3101 Washington Rd,McMurray,PA,15317,40.291685,-80.104900,3.0,11,1,Hair Stylists;Hair Salons;Men's Hair Salons;Bl...
2,KQPW8lFf1y5BT2MxiSZ3QA,Western Motor Vehicle,None,"6025 N 27th Ave, Ste 1",Phoenix,AZ,85017,33.524903,-112.115310,1.5,18,1,Departments of Motor Vehicles;Public Services ...
3,8DShNS-LuFqpEWIp0HxijA,Sports Authority,None,"5000 Arizona Mills Cr, Ste 435",Tempe,AZ,85282,33.383147,-111.964725,3.0,9,0,Sporting Goods;Shopping
4,PfOCPjBrlQAnz__NXj9h_w,Brick House Tavern + Tap,None,581 Howe Ave,Cuyahoga Falls,OH,44221,41.119535,-81.475690,3.5,116,1,American (New);Nightlife;Bars;Sandwiches;Ameri...


In [5]:
yelp_hours.head()

,business_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday
0,FYWN1wneV18bWNgQjJ2GNg,07:30-17:00,07:30-17:00,07:30-17:00,07:30-17:00,07:30-17:00,None,None
1,He-G7vWjzVUysIKrfNbPUQ,9:00-20:00,9:00-20:00,9:00-20:00,9:00-20:00,9:00-16:00,08:00-16:00,None
2,KQPW8lFf1y5BT2MxiSZ3QA,None,None,None,None,None,None,None
3,8DShNS-LuFqpEWIp0HxijA,10:00-21:00,10:00-21:00,10:00-21:00,10:00-21:00,10:00-21:00,10:00-21:00,11:00-19:00
4,PfOCPjBrlQAnz__NXj9h_w,11:00-01:00,11:00-01:00,11:00-01:00,11:00-01:00,11:00-01:00,11:00-02:00,11:00-00:00


In [6]:
yelp_reviews.head()

,business_name,review_id,user_id,stars,review_date,review_text,funny,useful,cool
0,"""Thai Pan Fresh Exotic Cuisine""",0ESSqLfOae77muWTv_zUqA,r-t7IiTSD0QZdt8lOUCqeQ,3,2011-02-11,Lately i have been feeling homesick for asian ...,1,1,1
1,"""Salon Lola""",VyAKIaj_Rmsf_ZCHcGJyUw,SS85hfTApRnbTPcJadra8A,5,2010-05-30,I love Marilo! She understands my hair type a...,0,0,0
2,"""Chipotle Mexican Grill""",DusrkpkTGPGkqK13xO1TZg,xAVu2pZ6nIvkdHh8vGs84Q,3,2011-11-26,Standard Chipotle fare - consistently good; no...,0,0,0
3,"""Sam's Deli""",A5FL-YYXnr2hnvXQbgkkrQ,Xuvj2Fq6c3mCmplVG7h21w,4,2013-10-10,Very good prices with quality ingredients.,0,0,0
4,"""Petco""",SzhIWfzLHUZfiAaVTtKHxw,jHH_UDpCnF-YTnveyoMTIA,1,2012-09-02,I went to this Petco a few times and was a bit...,0,1,0


## Concatenatations

### What are they?

If you think of your dataframes as rectangles in a 2D coordinate system concatenating means taking two or more rectangles and putting them next to each other so they touch on the concatenating axis.

We will use two types of concatenations, numpy and pandas.

The pandas version builds upon the numpy basics by incorporating ideas like indexes and keys but the main idea is the same.

The functions we will learn first are
- [np.hstack](https://docs.scipy.org/doc/numpy/reference/generated/numpy.dstack.html)
- [np.vstack](https://docs.scipy.org/doc/numpy/reference/generated/numpy.vstack.html)

In numpy lingo stack means concatenate. Pandas uses the terminology concat.

In [7]:
array1 = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

array1

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [8]:
array2 = np.array([[1, 0, 1], [1, 1, 1], [0, 1, 0]])

array2

array([[1, 0, 1],
       [1, 1, 1],
       [0, 1, 0]])

In [9]:
# hstack stands for horizontal stack (stack axis is parallel to Y axis)
print(np.hstack([array1, array2]))

[[1 2 3 1 0 1]
 [4 5 6 1 1 1]
 [7 8 9 0 1 0]]


In [10]:
# vstack stands for vertical stack (stack axis is parallel to X axis)
print(np.vstack([array1, array2]))

[[1 2 3]
 [4 5 6]
 [7 8 9]
 [1 0 1]
 [1 1 1]
 [0 1 0]]


Both stacking functions take a list of nd-arrays and return a single nd-array.

Here is another example this time using 1d-arrays and stacking to build a 2d-array.

In [11]:
row1 = np.array([3, -3, 3])
row2 = np.array([6, -9, 27])
row3 = np.array([9, -27, 81])

np.vstack([row1, row2, row3])

array([[  3,  -3,   3],
       [  6,  -9,  27],
       [  9, -27,  81]])

array([[  3,  -3,   3],
       [  6,  -9,  27],
       [  9, -27,  81]])

### Concatenating dataframes

Because dataframes are a bit more than 2d-arrays we can concatenate them as well.

We can use the numpy functions `hstack` and `vstack` but it is recommended to use `pd.concatenate` because of indexes.
- https://pandas.pydata.org/pandas-docs/stable/generated/pandas.concat.html

There are a lot of arguments but as always let's start small.

In [13]:
import pandas as pd
df1 = pd.DataFrame({
    "name": ["A", "B", "C", "D"],
    "age": [22, 18, 46, 81],
}, index=[0, 1, 2, 3])

df2 = pd.DataFrame({
    "name": ["G", "H", "I"],
    "age": [10, 20, 30]
}, index=[4, 5, 6])

print(df1)
print(df2)

  name  age
0    A   22
1    B   18
2    C   46
3    D   81
  name  age
4    G   10
5    H   20
6    I   30


In [14]:
# simplest possible use case will do a vstack
pd.concat([df1, df2])

,name,age
0,A,22
1,B,18
2,C,46
3,D,81
4,G,10
5,H,20
6,I,30


In [15]:
# Passing axis=1 will do hstack
pd.concat([df1, df2], axis=1)

,name,age,name,age
0,A,22.0,NaN,NaN
1,B,18.0,NaN,NaN
2,C,46.0,NaN,NaN
3,D,81.0,NaN,NaN
4,NaN,NaN,G,10.0
5,NaN,NaN,H,20.0
6,NaN,NaN,I,30.0


**Quick remark: np.hstack and np.vstack will throw an error if shapes do not match, pd.concatenate will extend the dataframes with null values depending on the join parameter.**

Passing the keys property will make the resulting dataframe have a [MultiIndex](https://pandas.pydata.org/pandas-docs/stable/advanced.html#advanced-hierarchical) whose first level is the value which describes the dataframe.

In [16]:
pd.concat([df1, df2], keys=['df_left', 'df_right'])

name  age
df_left  0    A   22
         1    B   18
         2    C   46
         3    D   81
df_right 4    G   10
         5    H   20
         6    I   30

When you concatenate on axis where the shapes do not match you should set the join paramater to one of its two values:
- "inner" which means take the intersection over the index axis of the dataframes being concatenated
- "outer" which means take the union over the index axis of the dataframes being concatenated

The default value is "outer" which gives the behaviour as described by the quick remark above.

In [17]:
pd.concat([df1, df2], axis=1, join='inner')

,name,age,name,age


In [18]:
pd.concat([df1, df2], axis=1, join='outer')

,name,age,name,age
0,A,22.0,NaN,NaN
1,B,18.0,NaN,NaN
2,C,46.0,NaN,NaN
3,D,81.0,NaN,NaN
4,NaN,NaN,G,10.0
5,NaN,NaN,H,20.0
6,NaN,NaN,I,30.0


The final important parameter to talk about is `ignore_index`.

If you pass `ignore_index=True` pandas will concatenate the dataframes without looking at the index.

Generally you do not need to use this but if you concatenate dataframes with different index types you will need this.

The [append](https://pandas.pydata.org/pandas-docs/stable/merging.html#concatenating-using-append) method is similar to concat in the sense that the following holds true

`df1.append(df2) = pd.concat([df1, df2])`

You should prefer concat almost always because it is more general.

#### Practical example 

List comprehension and pd.concat to get 1, 3, 5 stared rows from yelp_reviews dataset.

In [19]:
pd.concat([
           yelp_reviews[yelp_reviews.stars == stars] 
           for stars in ['1', '3', '5']
          ])

# another solution without using pd.concat
# yelp_reviews[yelp_reviews.stars.isin(['1', '3', '5'])]

,business_name,review_id,user_id,stars,review_date,review_text,funny,useful,cool
4,"""Petco""",SzhIWfzLHUZfiAaVTtKHxw,jHH_UDpCnF-YTnveyoMTIA,1,2012-09-02,I went to this Petco a few times and was a bit...,0,1,0
12,"""Canteen Modern Tequila Bar""",qnpnnZ_JrOLsIdtn5aB9Hw,_4lqpCYCqOQzbB6xQGGhrQ,1,2010-10-14,"Ok, so the grand opening hasn't even happened ...",1,3,1
60,"""Fairmont Scottsdale Princess""",pmXF1ajhiMietpARXXX-iw,PBdpLP3yMHV79Y7EjuBY4g,1,2010-11-25,We went for the Thanksgiving Brunch at the LV ...,0,0,0
71,"""Gordon Biersch Brewery Restaurant""",avwF0OJFhNxu-s95JDhS7A,ZAXpsNT5fi4csd86AXVqPw,1,2013-09-15,Got food poisoning off a simple Caesar salad. ...,0,2,0
79,"""Golden Buddha""",I7wxA8sjH9B5_4tTr5cNyQ,Z3qVl-1JgYnAe4py6VC5ug,1,2010-01-09,"First of all, I visited this place many times ...",0,1,0
...,...,...,...,...,...,...,...,...,...
16384,"""Cafeteria on Thomas""",Dsby7RQWi_pBaLOqdUwhfQ,HestW1GKSkmUs_vpnauE_w,5,2013-11-05,"Love this place, love this place, love this pl...",0,0,0
16386,"""Green New American Vegetarian""",0vPd2APapf1k6zkH-afhQw,At7cNBShE1OErhavh4Q9Hg,5,2012-10-03,"We've gone here a lot the past couple months, ...",0,0,0
16387,"""Taquitos Jalisco""",psGkX3ysOg_VD2vSPk9cEg,jx104chvz7NbkVytXldUGg,5,2013-06-14,Thanks to Mark R for introducing me to this pl...,3,4,4
16388,"""Grimaldi's Pizzeria""",66h5dw3U5t3Pe_q1_33JhQ,hYph1O9SCO3B5tFmqzdOSA,5,2009-05-05,"My 2nd favorite: grilled chicken, sundried to...",1,1,1


## Merges and Joins
- https://pandas.pydata.org/pandas-docs/stable/merging.html#database-style-dataframe-joining-merging

pandas can perform relational joins in memory using the `merge` method which we discuss for the remainder of this lesson.

Each dataframe also has a `join` method but the following holds true (for details see the docs)

`df1.join(df2) = pd.merge(df1, df2, left_index=True, right_index=True, how="left")`

By default merge does an inner join, while [`pd.DataFrame.join`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.join.html) does a left join but these are configurable with the how parameter.

The most imporant parameter of `pd.merge` is the choice of join type in the parameter called `how`.
- "inner" which corresponds to INNER JOIN in SQL
- "outer" which corresponds to FULL OUTER JOIN in SQL
- "left" which corresponds to LEFT OUTER JOIN in SQL
- "right" which corresponds to RIGHT OUTER JOIN in SQL

Second most important choice is what to join on.
- Column or a list of columns (e.g. ["bussiness_id"])
- Index 

You can choose either of them for both left and right(e.g. columns for left and index for right, or columns for both or indices for both)

*By default merge will work with indexes*

#### Example 1 of merge function

An example of inner join between yelp_business and yelp_hours on business_id column.

In [20]:
pd.merge(yelp_business, yelp_hours, how="inner", left_on=["business_id"], right_on=["business_id"])

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,monday,tuesday,wednesday,thursday,friday,saturday,sunday
0,FYWN1wneV18bWNgQjJ2GNg,Dental by Design,None,"4855 E Warner Rd, Ste B9",Ahwatukee,AZ,85044,33.330690,-111.978599,4.0,22,1,Dentists;General Dentistry;Health & Medical;Or...,07:30-17:00,07:30-17:00,07:30-17:00,07:30-17:00,07:30-17:00,None,None
1,He-G7vWjzVUysIKrfNbPUQ,Stephen Szabo Salon,None,3101 Washington Rd,McMurray,PA,15317,40.291685,-80.104900,3.0,11,1,Hair Stylists;Hair Salons;Men's Hair Salons;Bl...,9:00-20:00,9:00-20:00,9:00-20:00,9:00-20:00,9:00-16:00,08:00-16:00,None
2,KQPW8lFf1y5BT2MxiSZ3QA,Western Motor Vehicle,None,"6025 N 27th Ave, Ste 1",Phoenix,AZ,85017,33.524903,-112.115310,1.5,18,1,Departments of Motor Vehicles;Public Services ...,None,None,None,None,None,None,None
3,8DShNS-LuFqpEWIp0HxijA,Sports Authority,None,"5000 Arizona Mills Cr, Ste 435",Tempe,AZ,85282,33.383147,-111.964725,3.0,9,0,Sporting Goods;Shopping,10:00-21:00,10:00-21:00,10:00-21:00,10:00-21:00,10:00-21:00,10:00-21:00,11:00-19:00
4,PfOCPjBrlQAnz__NXj9h_w,Brick House Tavern + Tap,None,581 Howe Ave,Cuyahoga Falls,OH,44221,41.119535,-81.475690,3.5,116,1,American (New);Nightlife;Bars;Sandwiches;Ameri...,11:00-01:00,11:00-01:00,11:00-01:00,11:00-01:00,11:00-01:00,11:00-02:00,11:00-00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174562,ALV5R8NkZ1KGOZeuZl3u0A,Whitby Toyota,None,1025 Dundas Street W,Whitby,ON,L1P 1Z1,43.873460,-78.968033,4.0,4,1,Car Dealers;Automotive,07:00-18:00,07:00-18:00,07:00-18:00,07:00-18:00,07:00-18:00,08:00-16:00,None
174563,gRGalHVu6BcaUDIAGVW_xQ,Village Auto Body,None,3957 Brecksville Rd,Richfield,OH,44286,41.243385,-81.636212,5.0,3,1,Body Shops;Automotive,08:00-17:00,08:00-17:00,08:00-17:00,08:00-17:00,08:00-17:00,9:00-12:00,None
174564,XXvZBIHoJBU5d6-a-oyMWQ,AAM,None,"1600 W Broadway Rd, Ste 200",Tempe,AZ,85282,33.407914,-111.965098,1.5,19,1,Home Services;Property Management;Real Estate,08:30-17:00,08:30-17:00,08:30-17:00,08:30-17:00,08:30-17:00,None,None
174565,lNpPGgM96nPIYM1shxciHg,Bronze Beauty Spray Tanning,None,"300 Camp Horne Rd, Ste 250",Pittsburgh,PA,15202,40.517724,-80.091466,5.0,14,1,Spray Tanning;Tanning;Beauty & Spas,10:00-20:00,10:00-20:00,10:00-20:00,10:00-20:00,10:00-18:00,11:00-14:00,12:00-14:00


#### Example 2 of merge function

If the join columns are named the same in both dataframes you can use the `on` parameter and write less code.

The result of this join and the join right above is the same.

In [21]:
pd.merge(yelp_business, yelp_hours, how="inner", on=["business_id"])

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,monday,tuesday,wednesday,thursday,friday,saturday,sunday
0,FYWN1wneV18bWNgQjJ2GNg,Dental by Design,None,"4855 E Warner Rd, Ste B9",Ahwatukee,AZ,85044,33.330690,-111.978599,4.0,22,1,Dentists;General Dentistry;Health & Medical;Or...,07:30-17:00,07:30-17:00,07:30-17:00,07:30-17:00,07:30-17:00,None,None
1,He-G7vWjzVUysIKrfNbPUQ,Stephen Szabo Salon,None,3101 Washington Rd,McMurray,PA,15317,40.291685,-80.104900,3.0,11,1,Hair Stylists;Hair Salons;Men's Hair Salons;Bl...,9:00-20:00,9:00-20:00,9:00-20:00,9:00-20:00,9:00-16:00,08:00-16:00,None
2,KQPW8lFf1y5BT2MxiSZ3QA,Western Motor Vehicle,None,"6025 N 27th Ave, Ste 1",Phoenix,AZ,85017,33.524903,-112.115310,1.5,18,1,Departments of Motor Vehicles;Public Services ...,None,None,None,None,None,None,None
3,8DShNS-LuFqpEWIp0HxijA,Sports Authority,None,"5000 Arizona Mills Cr, Ste 435",Tempe,AZ,85282,33.383147,-111.964725,3.0,9,0,Sporting Goods;Shopping,10:00-21:00,10:00-21:00,10:00-21:00,10:00-21:00,10:00-21:00,10:00-21:00,11:00-19:00
4,PfOCPjBrlQAnz__NXj9h_w,Brick House Tavern + Tap,None,581 Howe Ave,Cuyahoga Falls,OH,44221,41.119535,-81.475690,3.5,116,1,American (New);Nightlife;Bars;Sandwiches;Ameri...,11:00-01:00,11:00-01:00,11:00-01:00,11:00-01:00,11:00-01:00,11:00-02:00,11:00-00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174562,ALV5R8NkZ1KGOZeuZl3u0A,Whitby Toyota,None,1025 Dundas Street W,Whitby,ON,L1P 1Z1,43.873460,-78.968033,4.0,4,1,Car Dealers;Automotive,07:00-18:00,07:00-18:00,07:00-18:00,07:00-18:00,07:00-18:00,08:00-16:00,None
174563,gRGalHVu6BcaUDIAGVW_xQ,Village Auto Body,None,3957 Brecksville Rd,Richfield,OH,44286,41.243385,-81.636212,5.0,3,1,Body Shops;Automotive,08:00-17:00,08:00-17:00,08:00-17:00,08:00-17:00,08:00-17:00,9:00-12:00,None
174564,XXvZBIHoJBU5d6-a-oyMWQ,AAM,None,"1600 W Broadway Rd, Ste 200",Tempe,AZ,85282,33.407914,-111.965098,1.5,19,1,Home Services;Property Management;Real Estate,08:30-17:00,08:30-17:00,08:30-17:00,08:30-17:00,08:30-17:00,None,None
174565,lNpPGgM96nPIYM1shxciHg,Bronze Beauty Spray Tanning,None,"300 Camp Horne Rd, Ste 250",Pittsburgh,PA,15202,40.517724,-80.091466,5.0,14,1,Spray Tanning;Tanning;Beauty & Spas,10:00-20:00,10:00-20:00,10:00-20:00,10:00-20:00,10:00-18:00,11:00-14:00,12:00-14:00


#### Example 3 of merge function

You can also join on indexes.

First we must make the bussiness_id column the index in both dataframes (see [set_index](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.set_index.html) for more info).

Then we join on index.

In [22]:
pd.merge(left=yelp_business.set_index(["business_id"]),
         right=yelp_hours.set_index(["business_id"]),
         how="inner",
         left_index=True,
         right_index=True)

,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,monday,tuesday,wednesday,thursday,friday,saturday,sunday
business_id,,,,,,,,,,,,,,,,,,,
FYWN1wneV18bWNgQjJ2GNg,Dental by Design,None,"4855 E Warner Rd, Ste B9",Ahwatukee,AZ,85044,33.330690,-111.978599,4.0,22,1,Dentists;General Dentistry;Health & Medical;Or...,07:30-17:00,07:30-17:00,07:30-17:00,07:30-17:00,07:30-17:00,None,None
He-G7vWjzVUysIKrfNbPUQ,Stephen Szabo Salon,None,3101 Washington Rd,McMurray,PA,15317,40.291685,-80.104900,3.0,11,1,Hair Stylists;Hair Salons;Men's Hair Salons;Bl...,9:00-20:00,9:00-20:00,9:00-20:00,9:00-20:00,9:00-16:00,08:00-16:00,None
KQPW8lFf1y5BT2MxiSZ3QA,Western Motor Vehicle,None,"6025 N 27th Ave, Ste 1",Phoenix,AZ,85017,33.524903,-112.115310,1.5,18,1,Departments of Motor Vehicles;Public Services ...,None,None,None,None,None,None,None
8DShNS-LuFqpEWIp0HxijA,Sports Authority,None,"5000 Arizona Mills Cr, Ste 435",Tempe,AZ,85282,33.383147,-111.964725,3.0,9,0,Sporting Goods;Shopping,10:00-21:00,10:00-21:00,10:00-21:00,10:00-21:00,10:00-21:00,10:00-21:00,11:00-19:00
PfOCPjBrlQAnz__NXj9h_w,Brick House Tavern + Tap,None,581 Howe Ave,Cuyahoga Falls,OH,44221,41.119535,-81.475690,3.5,116,1,American (New);Nightlife;Bars;Sandwiches;Ameri...,11:00-01:00,11:00-01:00,11:00-01:00,11:00-01:00,11:00-01:00,11:00-02:00,11:00-00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ALV5R8NkZ1KGOZeuZl3u0A,Whitby Toyota,None,1025 Dundas Street W,Whitby,ON,L1P 1Z1,43.873460,-78.968033,4.0,4,1,Car Dealers;Automotive,07:00-18:00,07:00-18:00,07:00-18:00,07:00-18:00,07:00-18:00,08:00-16:00,None
gRGalHVu6BcaUDIAGVW_xQ,Village Auto Body,None,3957 Brecksville Rd,Richfield,OH,44286,41.243385,-81.636212,5.0,3,1,Body Shops;Automotive,08:00-17:00,08:00-17:00,08:00-17:00,08:00-17:00,08:00-17:00,9:00-12:00,None
XXvZBIHoJBU5d6-a-oyMWQ,AAM,None,"1600 W Broadway Rd, Ste 200",Tempe,AZ,85282,33.407914,-111.965098,1.5,19,1,Home Services;Property Management;Real Estate,08:30-17:00,08:30-17:00,08:30-17:00,08:30-17:00,08:30-17:00,None,None


#### The suffixes parameter

The `suffixes` parameter is a tuple made of two strings.
- The first element of this tuple is the suffix for columns from the left dataframe
- The second element of this tuple is the suffix for columns from the right dataframe

The suffix is applied only to columns which have identical names and are not join keys.

For example
- d1 has columns ["key_id", "first_name", "age"]
- d2 has columns ["key_id", "last_name", "age"]

`pd.merge(d1, d2, on="key_id", suffixes=("_D1", "_D2"))`

The resulting dataframe will have the following columns
- ["key_id", "first_name", "last_name", "age_D1", "age_D2"]

Because age is present in both data frames and we must preserve both columns the solution is to rename both of them by appending a suffix string.

The default suffixes are ("_x", "_y") so you don't have to pass this paramater.

For no good reason other than to trouble you dear reader the `pd.DataFrame.join` method does not have a suffixes paramater but two paramaters:
- lsuffix which stands for left suffix
- rsuffix which stands for right suffix

The idea is still the same though.

#### Example 4. Full outer join in pandas.

Because this a full outer join all join keys are present (letters A to I).

In [23]:
pd.merge(df1, df2, how="outer", on=["name"])

,name,age_x,age_y
0,A,22.0,NaN
1,B,18.0,NaN
2,C,46.0,NaN
3,D,81.0,NaN
4,G,NaN,10.0
5,H,NaN,20.0
6,I,NaN,30.0


#### Example 5. Left outer join in pandas.

Because this a left outer join only keys from the left dataframe are present (A, B, C, D).

In [24]:
pd.merge(df1, df2, how="left", on=["name"], suffixes=("_df1", "_df2"))

,name,age_df1,age_df2
0,A,22,NaN
1,B,18,NaN
2,C,46,NaN
3,D,81,NaN


#### Example 1 of using the join function

Here we use a left outer join because we want a list of all bussiness irregardles if they have an entry in the hours table. If they don't have an entry all columns (Monday to Sunday) will be set to null which is the semantics of left join.

In [25]:
yelp_business.set_index("business_id", inplace=True)

yelp_hours.set_index("business_id", inplace=True)

yelp_business.join(yelp_hours)

,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,monday,tuesday,wednesday,thursday,friday,saturday,sunday
business_id,,,,,,,,,,,,,,,,,,,
FYWN1wneV18bWNgQjJ2GNg,Dental by Design,None,"4855 E Warner Rd, Ste B9",Ahwatukee,AZ,85044,33.330690,-111.978599,4.0,22,1,Dentists;General Dentistry;Health & Medical;Or...,07:30-17:00,07:30-17:00,07:30-17:00,07:30-17:00,07:30-17:00,None,None
He-G7vWjzVUysIKrfNbPUQ,Stephen Szabo Salon,None,3101 Washington Rd,McMurray,PA,15317,40.291685,-80.104900,3.0,11,1,Hair Stylists;Hair Salons;Men's Hair Salons;Bl...,9:00-20:00,9:00-20:00,9:00-20:00,9:00-20:00,9:00-16:00,08:00-16:00,None
KQPW8lFf1y5BT2MxiSZ3QA,Western Motor Vehicle,None,"6025 N 27th Ave, Ste 1",Phoenix,AZ,85017,33.524903,-112.115310,1.5,18,1,Departments of Motor Vehicles;Public Services ...,None,None,None,None,None,None,None
8DShNS-LuFqpEWIp0HxijA,Sports Authority,None,"5000 Arizona Mills Cr, Ste 435",Tempe,AZ,85282,33.383147,-111.964725,3.0,9,0,Sporting Goods;Shopping,10:00-21:00,10:00-21:00,10:00-21:00,10:00-21:00,10:00-21:00,10:00-21:00,11:00-19:00
PfOCPjBrlQAnz__NXj9h_w,Brick House Tavern + Tap,None,581 Howe Ave,Cuyahoga Falls,OH,44221,41.119535,-81.475690,3.5,116,1,American (New);Nightlife;Bars;Sandwiches;Ameri...,11:00-01:00,11:00-01:00,11:00-01:00,11:00-01:00,11:00-01:00,11:00-02:00,11:00-00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ALV5R8NkZ1KGOZeuZl3u0A,Whitby Toyota,None,1025 Dundas Street W,Whitby,ON,L1P 1Z1,43.873460,-78.968033,4.0,4,1,Car Dealers;Automotive,07:00-18:00,07:00-18:00,07:00-18:00,07:00-18:00,07:00-18:00,08:00-16:00,None
gRGalHVu6BcaUDIAGVW_xQ,Village Auto Body,None,3957 Brecksville Rd,Richfield,OH,44286,41.243385,-81.636212,5.0,3,1,Body Shops;Automotive,08:00-17:00,08:00-17:00,08:00-17:00,08:00-17:00,08:00-17:00,9:00-12:00,None
XXvZBIHoJBU5d6-a-oyMWQ,AAM,None,"1600 W Broadway Rd, Ste 200",Tempe,AZ,85282,33.407914,-111.965098,1.5,19,1,Home Services;Property Management;Real Estate,08:30-17:00,08:30-17:00,08:30-17:00,08:30-17:00,08:30-17:00,None,None


#### Example 2 of using the join function.

Using left outer join where the right dataframe is an empty dataframe you can add a lot of new columns to your existing dataframe by default filled with NaN.

Here we would like to add "address_street_number" and "address_street" to yelp_business.

This is not very useful here because we have little columns but this combined with `pd.DataFrame.fillna` method could make coding some algorithms an easier endeavour.

In [26]:
dummy_empty_df = pd.DataFrame(columns=["address_street_number", "address_street"])

yelp_business.join(dummy_empty_df)

,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,address_street_number,address_street
business_id,,,,,,,,,,,,,,
FYWN1wneV18bWNgQjJ2GNg,Dental by Design,None,"4855 E Warner Rd, Ste B9",Ahwatukee,AZ,85044,33.330690,-111.978599,4.0,22,1,Dentists;General Dentistry;Health & Medical;Or...,NaN,NaN
He-G7vWjzVUysIKrfNbPUQ,Stephen Szabo Salon,None,3101 Washington Rd,McMurray,PA,15317,40.291685,-80.104900,3.0,11,1,Hair Stylists;Hair Salons;Men's Hair Salons;Bl...,NaN,NaN
KQPW8lFf1y5BT2MxiSZ3QA,Western Motor Vehicle,None,"6025 N 27th Ave, Ste 1",Phoenix,AZ,85017,33.524903,-112.115310,1.5,18,1,Departments of Motor Vehicles;Public Services ...,NaN,NaN
8DShNS-LuFqpEWIp0HxijA,Sports Authority,None,"5000 Arizona Mills Cr, Ste 435",Tempe,AZ,85282,33.383147,-111.964725,3.0,9,0,Sporting Goods;Shopping,NaN,NaN
PfOCPjBrlQAnz__NXj9h_w,Brick House Tavern + Tap,None,581 Howe Ave,Cuyahoga Falls,OH,44221,41.119535,-81.475690,3.5,116,1,American (New);Nightlife;Bars;Sandwiches;Ameri...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ALV5R8NkZ1KGOZeuZl3u0A,Whitby Toyota,None,1025 Dundas Street W,Whitby,ON,L1P 1Z1,43.873460,-78.968033,4.0,4,1,Car Dealers;Automotive,NaN,NaN
gRGalHVu6BcaUDIAGVW_xQ,Village Auto Body,None,3957 Brecksville Rd,Richfield,OH,44286,41.243385,-81.636212,5.0,3,1,Body Shops;Automotive,NaN,NaN
XXvZBIHoJBU5d6-a-oyMWQ,AAM,None,"1600 W Broadway Rd, Ste 200",Tempe,AZ,85282,33.407914,-111.965098,1.5,19,1,Home Services;Property Management;Real Estate,NaN,NaN


## Exercises

#### Exercise #1

Find all bussiness which are from 'AZ', 'PA', 'OH', 'BW', 'NV' using concatenation.

In [0]:
yelp_business = get_dataset("yelp_business")
yelp_hours    = get_dataset("yelp_business_hours")
yelp_reviews  = get_dataset("yelp_reviews")

In [45]:
yelp_business.head(1)

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
0,FYWN1wneV18bWNgQjJ2GNg,Dental by Design,None,"4855 E Warner Rd, Ste B9",Ahwatukee,AZ,85044,33.33069,-111.978599,4.0,22,1,Dentists;General Dentistry;Health & Medical;Or...


In [46]:
yelp_hours.head(1)

,business_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday
0,FYWN1wneV18bWNgQjJ2GNg,07:30-17:00,07:30-17:00,07:30-17:00,07:30-17:00,07:30-17:00,None,None


In [47]:
yelp_reviews.head(1)

,business_name,review_id,user_id,stars,review_date,review_text,funny,useful,cool
0,Thai Pan Fresh Exotic Cuisine,0ESSqLfOae77muWTv_zUqA,r-t7IiTSD0QZdt8lOUCqeQ,3,2011-02-11,Lately i have been feeling homesick for asian ...,1,1,1


In [59]:
pd.concat([yelp_business,yelp_reviews],ignore_index=True)['city']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


0              Ahwatukee
1               McMurray
2                Phoenix
3                  Tempe
4         Cuyahoga Falls
               ...      
190952               NaN
190953               NaN
190954               NaN
190955               NaN
190956               NaN
Name: city, Length: 190957, dtype: object

#### Exercise #2

Find all bussiness which are from 'AZ', 'PA', 'OH', 'BW', 'NV' using inner join.

Hint: You can use a temporary dataframe.

In [0]:
tmp=pd.merge(left=yelp_business, right=yelp_reviews, how='inner', left_on=['name'], right_on=['business_name'], sort=True)

In [54]:
tmp

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars_x,review_count,is_open,categories,business_name,review_id,user_id,stars_y,review_date,review_text,funny,useful,cool
0,4z-QW_f3RwCAxHB5fd58TA,#1Brothers Pizza,None,"16995 W Greenway Rd, Ste 104",Surprise,AZ,85388,33.623660,-112.424707,3.0,60,1,Restaurants;Pizza,#1Brothers Pizza,GzZXkJTUaFPtIW9Bv5tJIw,KJo0cru2sBJyufnGkdHgHA,1,2012-03-03,This place sucks balls. I can't believe it is ...,1,0,1
1,V1wBDq0zqq2dlyQvzD5PaA,101 Bistro,None,101 N 1st Ave,Phoenix,AZ,85003,33.449799,-112.074714,3.5,7,0,American (New);Mediterranean;Restaurants;Greek,101 Bistro,qIupiJb1wtRWJzBWG0DYjQ,sEWeeq41k4ohBz4jS_iGRw,3,2008-04-25,Gyro and the ground beef Fatda (sp?) pita were...,1,0,2
2,bs07-hLuz0dq06gIUwR5mw,1130 The Restaurant,None,"455 N 3rd St, Ste 1130",Phoenix,AZ,85004,33.452872,-112.068550,3.5,204,1,American (Traditional);Steakhouses;American (N...,1130 The Restaurant,0x6sz5tnTB5WtEq4M-CwpA,LRuMicnZTzPfG6xFwwPSmA,1,2009-08-16,I USE to love 1130 when the service and manage...,0,1,0
3,bs07-hLuz0dq06gIUwR5mw,1130 The Restaurant,None,"455 N 3rd St, Ste 1130",Phoenix,AZ,85004,33.452872,-112.068550,3.5,204,1,American (Traditional);Steakhouses;American (N...,1130 The Restaurant,JIvwfKAHaFI5t9rjDJqL7w,e5tC3XMwxHcBPuddEN8wnw,3,2009-09-11,"Although I did not eat at 1130, I visited the ...",0,0,0
4,bs07-hLuz0dq06gIUwR5mw,1130 The Restaurant,None,"455 N 3rd St, Ste 1130",Phoenix,AZ,85004,33.452872,-112.068550,3.5,204,1,American (Traditional);Steakhouses;American (N...,1130 The Restaurant,B0pZ_YY65lfACHl4DzLb7Q,r-yd21BCYUKS3S1TFd7rLw,3,2013-05-25,I went to 1130 last night to celebrate my cous...,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179294,ErHMVkPstNyefb3ZU_z1Cw,zpizza,None,53 W Thomas Rd,Phoenix,AZ,85013,33.479992,-112.076718,3.5,247,1,Vegan;Gluten-Free;Restaurants;Pizza,zpizza,dWkvDQ1uJUXki0JTd1OyZw,#NAME?,1,2012-06-30,Over a week for horrible customer service...\n...,4,5,1
179295,ErHMVkPstNyefb3ZU_z1Cw,zpizza,None,53 W Thomas Rd,Phoenix,AZ,85013,33.479992,-112.076718,3.5,247,1,Vegan;Gluten-Free;Restaurants;Pizza,zpizza,2OD02P0WdL7y-5Qt6I4OrQ,tBvrnSCLSpUdCDm5w5GPkg,4,2008-06-17,Was introduced to this place at a street fair ...,0,2,0
179296,ErHMVkPstNyefb3ZU_z1Cw,zpizza,None,53 W Thomas Rd,Phoenix,AZ,85013,33.479992,-112.076718,3.5,247,1,Vegan;Gluten-Free;Restaurants;Pizza,zpizza,UJEPSoO6yNnR8kdneDy0rg,fSi-yrKtBD58h2vPxjNE1A,4,2010-12-01,We ordered 4 rusticas for delivery using their...,2,1,1
179297,ErHMVkPstNyefb3ZU_z1Cw,zpizza,None,53 W Thomas Rd,Phoenix,AZ,85013,33.479992,-112.076718,3.5,247,1,Vegan;Gluten-Free;Restaurants;Pizza,zpizza,#NAME?,c_XZwsOmiy4tc0Mmk_F2xw,4,2012-04-24,The pizza was crisp and delicious. The place i...,0,1,0


In [55]:
tmp.query("state in ('AZ', 'PA', 'OH', 'BW', 'NV')")['business_id']

0         4z-QW_f3RwCAxHB5fd58TA
1         V1wBDq0zqq2dlyQvzD5PaA
2         bs07-hLuz0dq06gIUwR5mw
3         bs07-hLuz0dq06gIUwR5mw
4         bs07-hLuz0dq06gIUwR5mw
                   ...          
179294    ErHMVkPstNyefb3ZU_z1Cw
179295    ErHMVkPstNyefb3ZU_z1Cw
179296    ErHMVkPstNyefb3ZU_z1Cw
179297    ErHMVkPstNyefb3ZU_z1Cw
179298    ErHMVkPstNyefb3ZU_z1Cw
Name: business_id, Length: 133225, dtype: object

#### Exercise #3

Make one giant table from yelp_business, yelp_reviews, yelp_business_hours.

Hint: use the merge function

In [60]:
yelp_business.head(1)

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
0,FYWN1wneV18bWNgQjJ2GNg,Dental by Design,None,"4855 E Warner Rd, Ste B9",Ahwatukee,AZ,85044,33.33069,-111.978599,4.0,22,1,Dentists;General Dentistry;Health & Medical;Or...


In [61]:
yelp_hours.head(1)

,business_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday
0,FYWN1wneV18bWNgQjJ2GNg,07:30-17:00,07:30-17:00,07:30-17:00,07:30-17:00,07:30-17:00,None,None


In [62]:
yelp_reviews.head(1)

,business_name,review_id,user_id,stars,review_date,review_text,funny,useful,cool
0,Thai Pan Fresh Exotic Cuisine,0ESSqLfOae77muWTv_zUqA,r-t7IiTSD0QZdt8lOUCqeQ,3,2011-02-11,Lately i have been feeling homesick for asian ...,1,1,1


In [0]:
tmp1=pd.merge(left=yelp_business, right=yelp_hours, left_on='business_id', right_on='business_id',indicator=False)

In [66]:
tmp1.head()

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,monday,tuesday,wednesday,thursday,friday,saturday,sunday
0,FYWN1wneV18bWNgQjJ2GNg,Dental by Design,None,"4855 E Warner Rd, Ste B9",Ahwatukee,AZ,85044,33.330690,-111.978599,4.0,22,1,Dentists;General Dentistry;Health & Medical;Or...,07:30-17:00,07:30-17:00,07:30-17:00,07:30-17:00,07:30-17:00,None,None
1,He-G7vWjzVUysIKrfNbPUQ,Stephen Szabo Salon,None,3101 Washington Rd,McMurray,PA,15317,40.291685,-80.104900,3.0,11,1,Hair Stylists;Hair Salons;Men's Hair Salons;Bl...,9:00-20:00,9:00-20:00,9:00-20:00,9:00-20:00,9:00-16:00,08:00-16:00,None
2,KQPW8lFf1y5BT2MxiSZ3QA,Western Motor Vehicle,None,"6025 N 27th Ave, Ste 1",Phoenix,AZ,85017,33.524903,-112.115310,1.5,18,1,Departments of Motor Vehicles;Public Services ...,None,None,None,None,None,None,None
3,8DShNS-LuFqpEWIp0HxijA,Sports Authority,None,"5000 Arizona Mills Cr, Ste 435",Tempe,AZ,85282,33.383147,-111.964725,3.0,9,0,Sporting Goods;Shopping,10:00-21:00,10:00-21:00,10:00-21:00,10:00-21:00,10:00-21:00,10:00-21:00,11:00-19:00
4,PfOCPjBrlQAnz__NXj9h_w,Brick House Tavern + Tap,None,581 Howe Ave,Cuyahoga Falls,OH,44221,41.119535,-81.475690,3.5,116,1,American (New);Nightlife;Bars;Sandwiches;Ameri...,11:00-01:00,11:00-01:00,11:00-01:00,11:00-01:00,11:00-01:00,11:00-02:00,11:00-00:00


In [0]:
tmp2=pd.merge(left=tmp1, right=yelp_reviews, left_on='name', right_on='business_name')

In [69]:
tmp2.head(2)

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars_x,review_count,is_open,categories,monday,tuesday,wednesday,thursday,friday,saturday,sunday,business_name,review_id,user_id,stars_y,review_date,review_text,funny,useful,cool
0,8DShNS-LuFqpEWIp0HxijA,Sports Authority,None,"5000 Arizona Mills Cr, Ste 435",Tempe,AZ,85282,33.383147,-111.964725,3.0,9,0,Sporting Goods;Shopping,10:00-21:00,10:00-21:00,10:00-21:00,10:00-21:00,10:00-21:00,10:00-21:00,11:00-19:00,Sports Authority,gmGo7oudxw8oFq1SJsXVhw,e8DXpXcwOPdP25hG2IGwog,5,2013-09-11,Huge store! Probably the biggest Sports Author...,0,0,1
1,8DShNS-LuFqpEWIp0HxijA,Sports Authority,None,"5000 Arizona Mills Cr, Ste 435",Tempe,AZ,85282,33.383147,-111.964725,3.0,9,0,Sporting Goods;Shopping,10:00-21:00,10:00-21:00,10:00-21:00,10:00-21:00,10:00-21:00,10:00-21:00,11:00-19:00,Sports Authority,gALIlKrgghasAhF9r9PHsQ,4_YfcfHkCem1onJFkItxjA,3,2008-01-08,Shopping for a jock strap in a former Albertso...,4,3,2


#### Exercise #4

Find the maximal review count from yelp_business and then using inner join find the business (or business) which have that review count.

In [70]:
yelp_business.head(1)

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
0,FYWN1wneV18bWNgQjJ2GNg,Dental by Design,None,"4855 E Warner Rd, Ste B9",Ahwatukee,AZ,85044,33.33069,-111.978599,4.0,22,1,Dentists;General Dentistry;Health & Medical;Or...


In [0]:
tmp3=yelp_business.sort_values(by='review_count', ascending=False)

In [79]:
tmp3.head()

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
97944,4JNXUYY8wbaaDmk3BPzlWw,Mon Ami Gabi,The Strip,3655 Las Vegas Blvd S,Las Vegas,NV,89109,36.112827,-115.172581,4.0,7361,1,French;Steakhouses;Restaurants;Breakfast & Brunch
119907,RESDUcs7fIiihp38-d6_6g,Bacchanal Buffet,The Strip,3570 S Las Vegas Blvd,Las Vegas,NV,89109,36.116113,-115.176222,4.0,7009,1,Sandwiches;Buffets;Restaurants;Breakfast & Bru...
69993,K7lWdNUhCbcnEvI0NhGewg,Wicked Spoon,The Strip,3708 Las Vegas Blvd S,Las Vegas,NV,89109,36.109538,-115.176170,3.5,5950,1,Buffets;Breakfast & Brunch;Restaurants
81212,cYwJA2A6I12KNkm2rtXd5g,Gordon Ramsay BurGR,The Strip,3667 Las Vegas Blvd S,Las Vegas,NV,89109,36.110724,-115.172169,4.0,5447,0,American (Traditional);Burgers;Restaurants
139699,DkYS3arLOhA8si5uUEmHOw,Earl of Sandwich,The Strip,3667 Las Vegas Blvd S,Las Vegas,NV,89109,36.108228,-115.171869,4.5,4869,1,Caterers;Sandwiches;Restaurants;Food Delivery ...


In [0]:
tmp3.rename(columns={'name':'business_name'},inplace=True)

In [88]:
tmp3.head(2)

,business_id,business_name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
97944,4JNXUYY8wbaaDmk3BPzlWw,Mon Ami Gabi,The Strip,3655 Las Vegas Blvd S,Las Vegas,NV,89109,36.112827,-115.172581,4.0,7361,1,French;Steakhouses;Restaurants;Breakfast & Brunch
119907,RESDUcs7fIiihp38-d6_6g,Bacchanal Buffet,The Strip,3570 S Las Vegas Blvd,Las Vegas,NV,89109,36.116113,-115.176222,4.0,7009,1,Sandwiches;Buffets;Restaurants;Breakfast & Bru...


In [90]:
yelp_reviews.head(1)

,business_name,review_id,user_id,stars,review_date,review_text,funny,useful,cool
0,Thai Pan Fresh Exotic Cuisine,0ESSqLfOae77muWTv_zUqA,r-t7IiTSD0QZdt8lOUCqeQ,3,2011-02-11,Lately i have been feeling homesick for asian ...,1,1,1


In [92]:
tmp3.dtypes

business_id       object
business_name     object
neighborhood      object
address           object
city              object
state             object
postal_code       object
latitude         float64
longitude        float64
stars            float64
review_count       int64
is_open            int64
categories        object
dtype: object

In [93]:
yelp_reviews.dtypes

business_name    object
review_id        object
user_id          object
stars            object
review_date      object
review_text      object
funny             int64
useful            int64
cool              int64
dtype: object

In [0]:
x=pd.concat([tmp3, yelp_reviews], axis=1)

In [98]:
x.iloc[97944]

business_id                                 4JNXUYY8wbaaDmk3BPzlWw
business_name                                         Mon Ami Gabi
neighborhood                                             The Strip
address                                      3655 Las Vegas Blvd S
city                                                     Las Vegas
state                                                           NV
postal_code                                                  89109
latitude                                                   36.1128
longitude                                                 -115.173
stars                                                            4
review_count                                                  7361
is_open                                                          1
categories       French;Steakhouses;Restaurants;Breakfast & Brunch
business_name                                                  NaN
review_id                                                     

#### Exercise #5

The yelp_reviews dataset has 14 rows where the stars column has a value of '?'.

We want to explore the businesses these reviews relate to.